In [1]:
### Library necessary to run this IPython Notebook
!pip install facebook-sdk
!pip install tqdm
!pip install requests
!pip install arrow

In [2]:
import pandas as pd
import facebook
import requests
import arrow
from tqdm import tqdm

In [9]:
ACCESS_TOKEN = 'Get token...'
PAGE_ID = 'brainstorm9'

In [10]:
graph = facebook.GraphAPI(access_token = ACCESS_TOKEN, version = '2.7')

In [11]:
page = graph.get_object(
    id = PAGE_ID,
    fields='name, about, category, fan_count, feed.limit(25){created_time, permalink_url, type, status_type, message, shares}'
)

In [15]:
arw = arrow.utcnow()
current_month = arw.month

def extract_feed_data(feed, rows = []):
    should_finish = False

    for post in tqdm(feed['data'], 'Extracting current month data...'):
        post_month = arrow.get(post['created_time']).month
        if(post_month != current_month):
            should_finish = True
            break
        
        likes = graph.get_object(str(post['id']) + '/likes?limit=0&summary=true')
        reactions = graph.get_object(str(post['id']) + '/reactions?limit=0&summary=true')
        comments = graph.get_object(str(post['id']) + '/comments?limit=0&summary=true')

        row_data = {
            'id': post['id'],
            'created_time': post['created_time'],
            'permalink_url': post['permalink_url'],
            'type': post['type'],
            'status_type': post['status_type'],
            'message': post['message'],
            'total_likes': likes['summary']['total_count'],
            'total_reactions': reactions['summary']['total_count'],
            'total_comments': comments['summary']['total_count'],
            'total_shares': post['shares']['count'] if 'shares' in post else 0
        }
        rows.append(row_data)

    if(should_finish):
        print('Month data complete!')
        return rows
    else:
        next_feed = requests.get(feed['paging']['next']).json()
        return extract_feed_data(next_feed, rows)

In [16]:
page_data = pd.DataFrame(extract_feed_data(page['feed']))


Extracting current month data...:  20%|██        | 5/25 [00:16<01:08,  3.43s/it]

Month data complete!


In [17]:
page_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 10 columns):
created_time       5 non-null object
id                 5 non-null object
message            5 non-null object
permalink_url      5 non-null object
status_type        5 non-null object
total_comments     5 non-null int64
total_likes        5 non-null int64
total_reactions    5 non-null int64
total_shares       5 non-null int64
type               5 non-null object
dtypes: int64(4), object(6)
memory usage: 480.0+ bytes


In [18]:
page_data.head()

,created_time,id,message,permalink_url,status_type,total_comments,total_likes,total_reactions,total_shares,type
0,2017-04-19T18:01:40+0000,277104869126156_717384965098142,Na próxima terça vamos promover uma palestra b...,https://www.facebook.com/IncubadoraInovaMetrop...,added_photos,2,4,4,3,photo
1,2017-04-12T18:28:56+0000,277104869126156_713650055471633,Daqui a pouco temos a palestra com o Marcelo A...,https://www.facebook.com/IncubadoraInovaMetrop...,added_photos,1,3,3,0,photo
2,2017-04-10T21:24:44+0000,277104869126156_712628078907164,Está lançado o edital de pintura das paredes d...,https://www.facebook.com/IncubadoraInovaMetrop...,added_photos,5,6,6,1,photo
3,2017-04-10T15:00:00+0000,277104869126156_713661245470514,Nessa segunda aconteceu o evento de boas-vinda...,https://www.facebook.com/IncubadoraInovaMetrop...,added_photos,0,7,7,0,photo
4,2017-04-08T18:58:54+0000,277104869126156_711420299027942,Na próxima quarta você não pode perder esse ba...,https://www.facebook.com/IncubadoraInovaMetrop...,added_photos,1,16,17,12,photo


In [ ]:
# References

# https://facebook-sdk.readthedocs.io/en/latest/api.html
# https://developers.facebook.com/docs/graph-api/reference/v2.7